# 主題： 音樂曲風辨識
### 成員：地理碩二 曹宇鈞、地理碩一 林子鈞、地理四 劉玫宜


#### 專題動機：


        音樂豐富了我們的生活，不管是百無聊賴時的精神慰藉，或是深夜獨自一人的心靈雞湯，不同風格的音樂總是能帶給我們許多不同的情緒。然而現在的音樂類型越趨多元與融合，辨識音樂風格本身並不是件容易的事情。
        近日盤據美國告示牌排行榜由Lil Nas X演唱的Old town road就產生了這首歌該被歸類在鄉村音樂或是嘻哈音樂的爭論。因此我們想到，如果能利用AI的方式，對音樂進行分類，或許會是件相當有趣的事。
        在手邊現有的資源中，有許多的音樂資料庫，而Youtube為了方便影音工作者進行影音工作，蒐集了大量的開放樂曲資料，而這也成為了我們這個專案能夠執行的契機。


#### 內容說明：
我們使用Youtube創作者工作室提供的音樂庫資料，作為進行曲風辨識的聲音素材。  <br/>
分類方面並不是本次專題的重點，因此我們直接沿用Youtube上現有的風格類別做為判斷依據。(當然後續原始碼釋出以後，使用者可以依照自己喜歡的分類喜好重新定義，並執行編碼)  <br/>
由於本專案資料量較大，且需要較好的電腦效能，故執行環境我們完全移植到Google Colab上面執行，環境設定會在稍後進行說明 。  <br/>
<br/>
#### 工作階段：
本專案的工作階段可以分成：「專案構想討論、資料處理、模型訓練、報告書面整理」四個階段。  <br/>

![流程圖](https://imgur.com/oXcEdgO.jpg)  




在**資料處理**部分，可以簡易分成三個階段  

#### 階段一：從Youtube Audio資料庫中撈取資料
1. 透過python selenium撰寫爬蟲爬取相關音訊資訊(json格式)  

2. 透過pandas讀取json檔案，以requests進行請求下載聲音檔案，並依照曲風進行資料夾分類。  

#### 階段二：將音樂資料預處理，轉換成model可直接讀取的np.array
本階段的預處理中，牽涉到音樂資料要處理成什麼格式。  
這部分我們參考了已經做過音樂分類的論文，發現多數人在進行音訊處理會使用 「librosa」這個package。 <br/>
在 librosa 當中有直接可以對音訊進行傅立葉轉換的函數，以及將傅立葉轉換之後的音訊資料，取出梅爾頻譜進行分析的函數。

#### 階段三：將np.array另存，並轉成可在model中使用的格式<br/>


# Google Drive環境設定

## 將個人google drive設定為Colab的工作區

In [ ]:
#專案開始只要執行一遍就好(之後同一個帳號不用再執行資料夾授權)
#這個程式碼會要求個人google 帳戶的SDK的權限。
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

E: Package 'python-software-properties' has no installation candidate
Selecting previously unselected package google-drive-ocamlfuse.
(Reading database ... 133872 files and directories currently installed.)
Preparing to unpack .../google-drive-ocamlfuse_0.7.18-0ubuntu1~ubuntu18.04.1_amd64.deb ...
Unpacking google-drive-ocamlfuse (0.7.18-0ubuntu1~ubuntu18.04.1) ...
Setting up google-drive-ocamlfuse (0.7.18-0ubuntu1~ubuntu18.04.1) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope

In [ ]:
#Linux的指令
#本指令也只需要執行一次就好
!mkdir -p drive #新增一個連結google drive資料的目錄
!google-drive-ocamlfuse drive  #連結到google drive

In [ ]:
# 查看當前工作位置與目錄清單
import os
print('Current work directory is : ',os.getcwd())
print('Directory list : ',os.listdir())

Current work directory is :  /content
Directory list :  ['.config', 'drive', 'adc.json', 'sample_data']


In [ ]:
#路徑設定
dirpath = "drive/AI_math"
if os.path.isdir(dirpath):
  print("existed")
  os.chdir(dirpath) #設定工作路徑
else:
  print("Unexisted")

existed


# Youtube檔案下載

## 使用的套件

In [ ]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sys, os, io, time
import requests, json

### 將音訊分類至各個風格的資料夾
* 整合json
* 依照曲風獲取download_url
* 下載至指定資料夾


![Imgur](https://i.imgur.com/29uDMrD.png)<br/>
![Imgur](https://i.imgur.com/uc44D0w.png) <br/>


#### json 整合

In [ ]:
json_folder = os.getcwd() + "\\YouTube-Audio-Library\\json"  #放置json檔案的位置
print(json_folder)

In [ ]:
# 把所有的json檔案整合成同一個json
rows_list = [] #儲存以下for迴圈的變數
for _, dirnames, filenames in os.walk(json_folder): #資料夾裡面的檔案逐項讀取
    # get all files in this folder
    for filename in filenames: #
        if(filename == 'all_data.json'):
            continue
            
        # read json with pandas
        df = pd.read_json(json_folder + "\\" + filename, encoding='utf-8')
        
        # audio information
        audios = df['tracks']
        for audio in audios:
            # append audio to list
            rows_list.append(audio)
        
        # create dataframe from list
        audios_df = pd.DataFrame(rows_list) #最後輸出的DataFrame(儲存所有的json的資料表)

In [ ]:
# DataFrame輸出成json檔案
audios_df.to_json(json_folder + "\\all_data.json")

#### 讀取剛剛整合好的json檔案，依照曲風獲得download_url，以及依照曲風創造資料夾

> 缩进块



In [ ]:
#資料讀取
audios_df = pd.read_json('all_data.json', encoding='utf-8')

#獲得曲風的屬性
genre = audios_df['genre']

# 計算每個曲風裡面有多少首歌
audios_df.groupby(['genre'],as_index=False)['genre'].agg({'count':'count'})
result = audios_df.groupby("genre")
result.count()

##### [支線]分類名稱略有差異→合併差異資料
* Country & Folk
* Country Folk
* Country and Folk
---
* Dance & Electronic
* Electronic
---
* Hip Hop & Rap
* Hip Hop

In [ ]:
def clearGenreClass(df, genre):
    result = df[df['genre'] == genre]
    df = df.drop(df.index[result.index.tolist()])
    
    return df
  
audios_df = clearGenreClass(audios_df, 'None')
audios_df = clearGenreClass(audios_df, 'World')

#輸出看一下結果
audios_df.groupby(['genre'],as_index=False)['genre'].agg({'count':'count'})

#檔案輸出(格式：.json)
audios_df.to_json(json_folder + "\\all_data.json")

#### 依照曲風新增資料夾

In [ ]:
#依照genre裡面的屬性新增list
dir_list = group['genre'].tolist()


#function ：在path底下創造新的資料夾
def createDirectory(path):
    base_dir = os.getcwd()

    try:  
        os.makedirs(path)
    except OSError:  
        print ("Creation of the directory %s failed" % path)
    else:  
        print ("Successfully created the directory %s" % path)      

    return base_dir + "\\" + path
 

 #根據dir_list創造資料夾路徑
for d in dir_list:
    createDirectory("genre/" + d)

#### 檔案下載

In [ ]:
base_path = os.getcwd() + "\\genre\\" #要下載的東西儲存的路徑設定到工作區底下的genre資料夾裡面
# print(base_path)
#資料下載的loop
for url, genre, title in zip(audios_df['download_url'], audios_df['genre'], audios_df['title']):
    filename = base_path + genre + "\\" + title + ".mp3" #檔案預計載入的路徑+檔名
    print(filename)
    #避免重複下載，檢查檔案是否存在
    if os.path.isfile(filename):
        print('existed')
    else:       
        # requests 向url請求檔案下在
        try:
          r = requests.get(url)
          with open(filename, 'wb') as f:  #寫入檔案
              f.write(r.content)
        except:
          print("request fail...") #如果寫入失敗的話
          with open('/path/to/file', 'r') as f:
            print(f.read())

# 資料讀取
### 設計說明
* 本部分與後續的模型設計，程式碼與結構參考自：  <br/>
 Hguimaraes (2018), [REPO] Music Genre classification on GTZAN dataset using CNNs. Github.  <br/>
 網址：https://github.com/Hguimaraes/gtzan.keras [2019.05.17]  <br/>
* LibROSA  package使用，參照 ：LibROSA — librosa 0.6.3 documentation  <br/>
網址：https://librosa.github.io/librosa/ [2019.05.17] <br/>

##需要的套件

In [ ]:
#import basic package
%matplotlib inline
%env KERAS_BACKED=tensorflow
import warnings
warnings.filterwarnings("ignore", category = FutureWarning)

import keras
import h5py
import librosa
import librosa.display
import itertools
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from collections import OrderedDict
from ipywidgets import interact
import math

from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Activation
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Dropout
from keras.layers import Flatten
from keras.layers import BatchNormalization

env: KERAS_BACKED=tensorflow


Using TensorFlow backend.


## Loading Data Function

### 將音訊陣列利用短時序傅立葉轉換，轉換成梅爾頻譜的格式
梅爾刻度根據一個轉換公式，在人耳可以聽見的範圍，從1000Hz，音量大於40分貝為基準，然後每500Hz去給定一個音高，根據不同頻率變化的幅度(amplitudes)轉換為梅爾頻譜後，再經過對數處理將數字差距拉大就能形成不同曲風的特徵值。<br/>
赫茲與梅爾頻譜的公式:<br/>
$m = 2595log_{10}\left ( 1+\frac{f}{700} \right )$<br/>

In [ ]:
def to_melspectrogram(songs, n_fft = 1024, hop_length = 512):
    # Transformation function
    melspec = lambda x: librosa.feature.melspectrogram(x, n_fft = n_fft,hop_length = hop_length)[:,:,np.newaxis]

    # map transformation of input songs to melspectrogram using log-scale
    tsongs = map(melspec, songs)
    return np.array(list(tsongs))

### 歌曲分割 (用於增加樣本數)
本函數只取每首歌前30秒，每3秒分為一個音檔，有1.5秒與前段重複。song_sample本次設定為660000，若要產生更多歌曲樣本，請更改song_sample的數字。<br/>

採樣率:每秒從連續訊號中提取並組成離散訊號的取樣個數，以赫茲（Hz）來表示。<br/>

本次取用的音訊採樣率為22050Hz(此為一般無線電廣播之採樣率)<br/>

計算公式:<br/>
一個音樂檔案所包含的聲波長度<br/>
$$22050(Hz)*30(sec) = 661500(Hz)$$<br/>
而函式設定3秒切割一個音檔，剩餘的音訊長度不足以成為一個音檔，因此song_sample取660000Hz<br/>


In [ ]:
def splitsongs(X, y, window = 0.1, overlap = 0.5):
    # Empty lists to hold our results
    temp_X = []
    temp_y = []

    # Get the input song array size
    xshape = X.shape[0]
    chunk = int(xshape*window)
    offset = int(chunk*(1.-overlap))
    
    # Split the song and create new ones on windows
    spsong = [X[i:i+chunk] for i in range(0, xshape - chunk + offset, offset)]
    for s in spsong:
        temp_X.append(s)
        temp_y.append(y)

    return np.array(temp_X), np.array(temp_y)

### 訓練資料預處理的檔案讀取函數

In [ ]:
def read_data(audios_df,data_list,np_ar1,np_ar2,read_list2,src_dir, genres, song_samples, spec_format, debug = True):    
    # Empty array of dicts with the processed features from all files
    arr_specs = []
    arr_genres = []
    read_list = []

    # Read files from the folders
    for x,_ in genres.items():
        folder = src_dir + x
        
        for root, subdirs, files in os.walk(folder):
            for file,idx in zip(files,range(0,len(files))):
                # Read the audio file
                file_name = folder + "/" + file
                #檢查重複
                if sum(data_list==file_name) >= 1:
                  print(file_name,'已經存在,跳過!!')
                  continue
                  
                else:
                  # Debug process
                  if debug:
                    print(idx,"Reading file: {}".format(file_name))
                  #防悲劇機制
                  t = ()
                  t = audios_df['len'][audios_df['title'] == str(file).replace(".mp3","")]
                  t = pd.DataFrame(t)
                  if len(t) > 1: #避免名字重複發生慘劇
                    for g,idx in zip(t.index,range(0,len(t),1)):
                      gg = audios_df['genre'][g]
                      if gg == x:
                        t = (float(t.reset_index()['len'][idx]))
                        break
                      else:
                        continue
                  else:
                        t = float(t['len'])
                  #Is audio over 30s                  
                  if t < 30:
                    print('不到30秒')
                    continue
                    
                  else:
                    signal, sr = librosa.load(file_name)
                    signal = signal[:song_samples]
                    print('load_done')
                    
                  # Convert to dataset of spectograms/melspectograms
                  signals, y = splitsongs(signal, genres[x])                
                  # Convert to "spec" representation
                  specs = spec_format(signals)
                  # Save files
                  arr_genres.extend(y)
                  arr_specs.extend(specs)
                  read_list.append(file_name)
                  #儲存中間過程的檔案，怕colab關起來

                  np.save(str(np_ar1), arr_specs) #music_strongth
                  np.save(str(np_ar2), arr_genres) #music class 
                  np.save(str(read_list2), read_list)
                
    return np.array(arr_specs), np.array(arr_genres)

## 檔案讀取與在模型裡面實作

In [ ]:
X = np.load('all_signal_list0_13.npy')
y = np.load('all_cat_list.npy')

In [ ]:
x = X[:313728]
sr=22050
melspec = librosa.feature.melspectrogram(x, sr, n_fft=1024, hop_length=512, n_mels=128)
logmelspec = librosa.power_to_db(melspec)
plt.figure(figsize=(16,8))
librosa.display.specshow(logmelspec, x_axis='time', y_axis='mel')
plt.colorbar()

In [ ]:
X = X.reshape(79097,128,129,1)
X.shape,y.shape

In [ ]:
y = to_categorical(y)
y.shape

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify = y)

print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

In [ ]:
# Histogram for train and test 
values, count = np.unique(np.argmax(y_train, axis=1), return_counts=True)
plt.bar(values, count)

values, count = np.unique(np.argmax(y_test, axis=1), return_counts=True)
plt.bar(values, count)

In [ ]:
# Model Definition
input_shape = X_train[0].shape
num_genres = 14

model = Sequential()
# Conv Block 1
model.add(Conv2D(16, kernel_size=(3, 3), strides=(1, 1),
                 activation='relu', input_shape=input_shape))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
# model.add(Dropout(0.25))

# Conv Block 2
model.add(Conv2D(32, (3, 3), strides=(1, 1), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
# model.add(Dropout(0.25))

# Conv Block 3
model.add(Conv2D(64, (3, 3), strides=(1, 1), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
# model.add(Dropout(0.25))

# Conv Block 4
model.add(Conv2D(128, (3, 3), strides=(1, 1), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
model.add(Dropout(0.25))

# Conv Block 5
model.add(Conv2D(64, (3, 3), strides=(1, 1), activation='relu'))
model.add(MaxPooling2D(pool_size=(4, 4), strides=(4, 4)))
# model.add(Dropout(0.25))

# MLP
model.add(Flatten())
model.add(Dense(num_genres, activation='softmax'))

model.summary()

In [ ]:
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adam(),
              metrics=['accuracy'])

hist = model.fit(X_train, y_train,
          batch_size=1000,
          epochs=5,
          verbose=1,
          validation_data=(X_test, y_test))

# score = model.evaluate(X_test, y_test, verbose=0)
# print("val_loss = {:.3f} and val_acc = {:.3f}".format(score[0], score[1]))